Solution:

---

---

In [ ]:
#include <iostream>
#include <string>
#include <vector>
#include <cmath>
#include <iomanip>
#include <format>    // For std::format (C++20)
#include <sstream>   // Fallback for string formatting

using namespace std;

class Solution {
private:
    // Count how many times a prime 'p' divides n! using Legendre's formula.
    // Example: count_factors_in_factorial(10, 2) = 8 (because 10! has 8 factors of 2).
    long long count_factors_in_factorial(int n, int p) {
        if (n < p) return 0;
        long long count = 0;
        long long power_of_p = p;
        while (power_of_p <= n) {
            count += n / power_of_p;   // Count multiples of p, p^2, p^3, ...
            // Prevent overflow when multiplying
            if (power_of_p > n / p) break;
            power_of_p *= p;
        }
        return count;
    }

    // Compute log10(n!) efficiently using the gamma function.
    // lgamma(n+1) = ln(n!), then convert to base-10 logarithm.
    long double log10_factorial(int n) {
        if (n <= 1) return 0.0;
        return lgammal(n + 1.0) / logl(10.0);
    }
    
    // Fast modular exponentiation (base^exp % mod).
    // Uses binary exponentiation with __int128 to avoid overflow.
    long long power(long long base, long long exp, long long mod) {
        long long res = 1;
        base %= mod;
        while (exp > 0) {
            if (exp % 2 == 1) 
                res = (static_cast<__int128>(res) * base) % mod;
            base = (static_cast<__int128>(base) * base) % mod;
            exp /= 2;
        }
        return res;
    }

public:
    string abbreviateProduct(int left, int right) {
        // --- Step 1: Factor counting & logarithm of product ---
        // Count powers of 2 and 5 separately (for trailing zeros).
        long long count2 = count_factors_in_factorial(right, 2) - count_factors_in_factorial(left - 1, 2);
        long long count5 = count_factors_in_factorial(right, 5) - count_factors_in_factorial(left - 1, 5);
        long double log_sum = log10_factorial(right) - log10_factorial(left - 1);
        
        // Trailing zeros come from matched pairs of 2 and 5.
        long long trailing_zeros = min(count2, count5);

        // --- Step 2: Compute last few digits of product ---
        // We compute product modulo large base (1e11) after removing 2s and 5s.
        long long suffix_mod = 100000000000LL; // 1e11
        long long suffix = 1;
        for (int i = left; i <= right; ++i) {
            int n = i;
            // Remove factors of 2 and 5 (already counted).
            while (n % 2 == 0) n /= 2;
            while (n % 5 == 0) n /= 5;
            suffix = (static_cast<__int128>(suffix) * n) % suffix_mod;
        }
        // Reintroduce the leftover powers of 2 and 5 (not forming trailing zeros).
        suffix = (static_cast<__int128>(suffix) * power(2, count2 - trailing_zeros, suffix_mod)) % suffix_mod;
        suffix = (static_cast<__int128>(suffix) * power(5, count5 - trailing_zeros, suffix_mod)) % suffix_mod;

        // --- Step 3: Decide whether to abbreviate or not ---
        long double num_digits_log = log_sum - trailing_zeros; 
        long long num_digits = floorl(num_digits_log) + 1; // total digits in product without trailing zeros
        
        if (num_digits > 10) {
            // For long results: keep prefix + suffix + scientific notation
            int prefix = powl(10, num_digits_log - floorl(num_digits_log) + 4);
            long long final_suffix_val = suffix % 100000; // last 5 digits
            
            // Example: "12345...67890e12"
            return std::format("{}...{:0>5}e{}", prefix, final_suffix_val, trailing_zeros);
            
            /*
            // Fallback using stringstream if std::format is not available
            stringstream ss;
            ss << prefix << "..." << setfill('0') << setw(5) << final_suffix_val << "e" << trailing_zeros;
            return ss.str();
            */
        } else {
            // For short results: return whole suffix with exponent.
            return std::format("{}e{}", suffix, trailing_zeros);
        }
    }
};


Test Case:

In [ ]:
Solution sol;

return 0;